In [2]:
import pprint
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse
from bs4.element import Comment
import urllib.request
import sys
import os
from os import path
from datetime import datetime
import csv

In [15]:
class CollegeCrawl(object):
    gap_Insecond=0
    max_Pages=10
    
    """
        collegename: name
        rooturl: www.university.edu
        prioritykeywords: ['apply','adimission'...] etc. if None then everth page 
        respectrobottxt: True
    """
    def __init__(self,_collegename, _rooturl, _prioritykeywords, _respectrobottxt=True):
        self.college=_collegename
        self.rootUrl=_rooturl
        self.priorityKeywords=_prioritykeywords
        self.respectRobottext=_respectrobottxt
        self.allUrls=[]
        self.visitedUrls=[]
        self.rejectedUrls=[]
        self.n_allUrls=0
        self.n_sitedUrls=0
        self.n_rejectedUrls=0
    
        

    """
        get all urls starting from rootUrl
    """
    def all_pages(self, base_url, links_only=True):
        response=requests.get(base_url, headers={'User-Agent':'Mozilla/5.0'})
        unique_urls={base_url}
        visited_urls=set()
        unables=set()
        while len(unique_urls)>len(visited_urls):
            soup=BeautifulSoup(response.text, 'html.parser')
            for link in soup.find_all('a'):
                try:
                    url=link['href']                
                    parsed_uri = urlparse(url )
                    if parsed_uri.netloc=='':
                        absolute_url=base_url+url    
                    elif parsed_uri.netloc==base_domain:
                        absolute_url=url
                    else:
                        continue
                    clean=re.sub(r'[_+!@#$?\\\s]+$', '', absolute_url)
                    clean=''.join(clean[0:11])+(''.join(clean[11:])).replace('//', '/')
                    clean=re.sub(r'[/\s]$','', clean)
                    unique_urls.add(clean)           
                except:
                    continue            
            unvisited_url=(unique_urls-visited_urls-unables).pop()
            visited_urls.add(unvisited_url)
            response=requests.get(unvisited_url)
            if links_only!=True and response.status_code== 200: #use a different routine to get text data from resonse. 
                handleOnePage(unvisited_url)
            else:
                unables.add(unvisited_url)  
        return unique_urls
    """
        read one page
    """
    def read_OneUrl(self, url):
        response=requests.get(url)  
        if response.status_code==200: 
            return self.get_pagetext(response)
        else: 
            return [[None, None, None, None]]
            
    def tag_visible(self, element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True 

    def get_pagetext(self, body):
        soup = BeautifulSoup(body.text, 'html.parser')
        texts = soup.findAll(text=True) 
        visible_texts = filter(self.tag_visible, texts)       
    
        return [ [t.parent.name,   
             t.parent.previousSibling.name if t.parent.previousSibling!=None else None, 
             t.nextSibling.name if t.nextSibling!=None else None,
             re.sub(r'[\s+\t]',' ',t) ]  for t in visible_texts if len(t.strip())>2] 
        
    
        
    """
        Save One Page
    """
    
    def save_OnePage(self,url,folder=None, filename=None, format='csv'):
        url=url.strip()
        content=self.read_OneUrl(url) #in format of (a,a,a,a)
     
        if folder==None:
            folder=os.getcwd()
        if path.isdir(folder)==False:
            print('folder doesnot exist')
            return
        if filename==None:
            filename=url.replace('.', '_dot_').replace('/', '_').replace(':', '_')+'_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")+'.csv'
    
        fullname=path.join(folder, filename)
        with open(fullname, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
            writer.writerow(['url', 'parent', 'ps', 'ns', 'text'])
            for lll in content: 
                lll.insert(0, url)
                writer.writerow(lll)  
        return fullname           
    

     

In [16]:
names=['UCLA','Yale','University of Washington','Stanford']
urls=['http://www.ucla.edu/','http://yale.edu/','http://washington.edu/','http://stanford.edu/']
k=['apply,admission']
for q in range(len(names)):
    c=CollegeCrawl(names[q],urls[q],k)
    a=c.all_pages(urls[q])
    for x in a:
        c.save_OnePage(x)

In [13]:
#response=requests.get("https://stanford.edu", None, headers={'User-Agent': "*"})
#response=requests.get("https://stanford.edu", headers={'User-Agent':'Mozilla/5.0'})
url = "http://www.mit.edu"
response = requests.get(url)
print("headers sent for URL {} is {}".format(url, response.request.headers))
print("response status code for URL {} is {}".format(url, response.status_code))
help(requests.get)

headers sent for URL http://www.mit.edu is {'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
response status code for URL http://www.mit.edu is 200
Help on function get in module requests.api:

get(url, params=None, **kwargs)
    Sends a GET request.
    
    :param url: URL for the new :class:`Request` object.
    :param params: (optional) Dictionary, list of tuples or bytes to send
        in the query string for the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response

